# API Client

> This is a client that can access an API Server that is running a remote host. Make sure to start the server before running client commands.



### Create a Client

In [9]:
from known.api.client import Client

# create a client
client = Client(server='localhost:8080', uid='bob') 
print(client.uid)

# NOTE:
# the client uid specifies a string that the server uses to allow access to the API
# this setting can be defined in server's configuration file in `config['uids']`

bob


### Check if server is up or not

In [10]:
# check connection to ther server, returns true if connection is up
if client.check():  print(f'server is up!')
else:               raise ValueError(f'server is down!')


server is up!


# Send requests

we can send 4 types of requests to API server based on the type of data that we need to send, these are `MESG`, `JSON`, `BYTE` and `FORM`

### MESG requests (strings)

In [11]:
response = client.send_mesg("Hi, this is a sample message.")

print(response)
print(response())

[✔️] | Type=MESG | Tag=default_handle | Content=<class 'str'>
default_response


### JSON requests (json serializable objects)

In [12]:
response = client.send_json(dict(name='alice', age=5, height=12.3, ls=[1,2.0,"A"]))

print(response)
print(response())

[✔️] | Type=MESG | Tag=default_handle | Content=<class 'str'>
default_response


### BYTE requests (bytes data)

In [13]:
response = client.send_byte(b'Hi, this is sample byte-data')

print(response)
print(response())

[✔️] | Type=MESG | Tag=default_handle | Content=<class 'str'>
default_response


### FORM requests (form data, including files)

In [17]:
from known.api.client import  Client, ClientForm
# NOTE: client form only support string type key-value pairs
# ...   anyother type will be converted to string
form = ClientForm(
    name = 'bob',  
    age=6,              #<--- NOTE: this will be converted to string
    height=15.03,       #<--- NOTE: this will be converted to string
    ls=[2,3.3,"B"],     #<--- NOTE: this will be converted to string
)\
.attach(alias='file1', name='overview.md', mime='markdown', handle='setup.py')\
.attach(alias='file2', name='readme.txt', mime='text', handle='README.md')
response = client.send_form(form)

print(response)
print(response())

[✔️] | Type=MESG | Tag=default_handle | Content=<class 'str'>
default_response


# Path Requests

In [19]:
# get the HOME view of the store
response = client.path_get()
print(response)
if response.ok:
    for i,(k,v) in enumerate(response().items()): print(f'\n#[{i}]\nPATH: {k}\nFILES: {v}')
else: print(response())

[✔️] | Type=H | Tag=/home/ava/Code/known/__api__ | Content=<class 'dict'>

#[0]
PATH: .
FILES: ['api.py']

#[1]
PATH: __pycache__
FILES: ['api.cpython-311.pyc']


In [20]:
# get the DIR view of the root folder
response = client.path_get(path='', save=None)
print(response)
if response.ok:
    for k,v in response().items(): print(f'{k}\t{v}')
else: print(response())

[✔️] | Type=D | Tag=. | Content=<class 'dict'>
base	.
files	{'api.py': 1.6}
folders	['__pycache__']


In [21]:
# get the DIR view of a folder - this folder does not exsist so it will return false
response = client.path_get(path='logdir', save=None)
print(response)
if response.ok:
    for k,v in response().items(): print(f'{k}\t{v}')
else: print(response())

[❌] | Type=None | Tag=None | Content=<class 'str'>
Response not ok 

Path not found: /home/ava/Code/known/__api__/logdir


In [22]:
# create a folder on server
response = client.path_set(path='logdir', item=None)
print(response)
print(response())

[✔️] | Type=M | Tag=logdir | Content=<class 'str'>
Folder created @ /home/ava/Code/known/__api__/logdir


In [24]:
# get the DIR view of a folder - this folder was created so it returns true
response = client.path_set(path='logdir', item=None)
print(response)
print(response())

[✔️] | Type=M | Tag=logdir | Content=<class 'str'>
Folder created @ /home/ava/Code/known/__api__/logdir


In [26]:
# create a file on server inside new folder
response = client.path_set(path='logdir/api.md', item='README.md')
print(response)
print(response())

[✔️] | Type=M | Tag=logdir/api.md | Content=<class 'str'>
File created @ /home/ava/Code/known/__api__/logdir/api.md


In [27]:
# create another folder inside the new folder
response = client.path_set(path='logdir/extra', item=None)
print(response)
print(response())

[✔️] | Type=M | Tag=logdir/extra | Content=<class 'str'>
Folder created @ /home/ava/Code/known/__api__/logdir/extra


In [29]:
# create 2 new files inside the nested folder

response = client.path_set(path='logdir/extra/a.md', item='README.md')
print(response)
print(response())

response = client.path_set(path='logdir/extra/b.txt', item='requirements.txt')
print(response)
print(response())

[✔️] | Type=M | Tag=logdir/extra/a.md | Content=<class 'str'>
File created @ /home/ava/Code/known/__api__/logdir/extra/a.md
[✔️] | Type=M | Tag=logdir/extra/b.txt | Content=<class 'str'>
File created @ /home/ava/Code/known/__api__/logdir/extra/b.txt


In [30]:
# Home view - to see all available paths on the store and the files inside them
response = client.path_get()
print(response)
if response.ok:
    for k,v in response().items(): print(f'{k}\t{v}')
else: print(response())

[✔️] | Type=H | Tag=/home/ava/Code/known/__api__ | Content=<class 'dict'>
.	['api.py']
__pycache__	['api.cpython-311.pyc']
logdir	['api.md']
logdir/extra	['b.txt', 'a.md']


In [31]:
# Dir view of the root folder (number beside the filenames indicate size in KB)
response = client.path_get(path='', save=None)
print(response)
if response.ok:
    for k,v in response().items(): print(f'{k}\t{v}')
else: print(response())

[✔️] | Type=D | Tag=. | Content=<class 'dict'>
base	.
files	{'api.py': 1.6}
folders	['logdir', '__pycache__']


In [32]:
# dir view of created folder
response = client.path_get(path='logdir', save=None)
print(response)
if response.ok:
    for k,v in response().items(): print(f'{k}\t{v}')
else: print(response())

[✔️] | Type=D | Tag=logdir | Content=<class 'dict'>
base	logdir
files	{'api.md': 4.45}
folders	['extra']


In [33]:
# dir view of created folder
response = client.path_get(path='logdir/extra', save=None)
print(response)
if response.ok:
    for k,v in response().items(): print(f'{k}\t{v}')
else: print(response())

[✔️] | Type=D | Tag=logdir/extra | Content=<class 'dict'>
base	logdir/extra
files	{'a.md': 4.45, 'b.txt': 0.76}
folders	[]


In [34]:
# download a file from server and save it at api.txt
response = client.path_get(path='logdir/extra/a.md', save='api.txt')
print(response)
print(response())

[✔️] | Type=F | Tag=a.md | Content=<class 'NoneType'>
None


In [35]:
# trying to download non-existing files
response = client.path_get(path='logdir/extra/x.md', save='api.txt')
print(response)
print(response())

[❌] | Type=None | Tag=None | Content=<class 'str'>
Response not ok 

Path not found: /home/ava/Code/known/__api__/logdir/extra/x.md


In [36]:
# create a new file on the root folder
response = client.path_set(path='api.txt', item='setup.py')
print(response)
print(response())

[✔️] | Type=M | Tag=api.txt | Content=<class 'str'>
File created @ /home/ava/Code/known/__api__/api.txt


In [37]:
# ... will overwrite when called again
response = client.path_set(path='api.txt', item='setup.py')
print(response)
print(response())

[✔️] | Type=M | Tag=api.txt | Content=<class 'str'>
File created @ /home/ava/Code/known/__api__/api.txt


In [38]:
# deleting a folder that is not empty without the recurvie flag
response = client.path_del(path='logdir', recursive=False)
print(response)
print(response())

[❌] | Type=None | Tag=None | Content=<class 'str'>
Response not ok 

Cannot delete folder at /home/ava/Code/known/__api__/logdir


In [39]:
# deleting a folder that is not empty with the recurvie flag
response = client.path_del(path='logdir', recursive=True)
print(response)
print(response())

[✔️] | Type=M | Tag=logdir | Content=<class 'str'>
Folder deleted @ /home/ava/Code/known/__api__/logdir


In [40]:
# deleting a file
response = client.path_del(path='api.txt', recursive=False)
print(response)
print(response())

[✔️] | Type=M | Tag=api.txt | Content=<class 'str'>
File deleted @ /home/ava/Code/known/__api__/api.txt
